# Proyecto Final Algebra Lineal 2017
---

El proyecto del curso consiste en la implementación del [Algortimo de Canny](https://es.wikipedia.org/wiki/Algoritmo_de_Canny) para la detección de bordes en imágenes y videos. Para ello, se le entrega al estudiante toda la implementación del algoritmo en [OpenCV](http://opencv.org/) y su objetivo es **implementar las funciones de OpenCV con matrices**. 

Las funciones a modificar serán las siguientes:
```python
cv2.GaussianBlur()
cv2.Sobel()
```

No es necesario que modifique más del código que se le solicita, pero puede hacer las modificaciones pertinentes si ese fuera el caso. Dentro del código se le indica en qué parte debe de ser modificado, por ejemplo:

```python
'''Modificar esta función'''
    sobel = cv2.Sobel(img, cv2.CV_64F, x, y, ksize=sobel_kernel)
``` 

Se recomienda buscar maneras de optimizar los cálculos de sus filtros, pues más adelante existe una aplicación en tiempo real donde es crucial que sus cálculos sean lo más rápidos posibles. Las dimensiones de los *kernels* no pueden ser modificadas.

## 0. Conceptos
Para poder modificar las funciones solicitadas es necesario que comprenda algunos conceptos, por lo cual es necesario que defina:

1. ¿En que consiste la convolución y cómo puede ser calculada? (Puede dar una definición matemática y/o ejemplos).

2. ¿Qué es el *kernel* de un filtro y para que se utiliza?

3. ¿En qué consiste un filtro Gaussiano?

4. ¿En qué consiste el Filtro Sobel? ¿para qué sirve?

3. Describa el algoritmo de detección de bordes Canny.

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import cv2

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

In [ ]:
def Mygauss_blur(img):
    
    kernel = np.array([[1, 2,3,2, 1],
                       [2, 7,11,7, 2],
                       [3, 11,17,11, 3],
                        [2, 7,11,7, 2],
                        [1, 2,3,2, 1],,dtype=np.float)
    r= img.shape[0]
    c= img.shape[1]
    gauss = np.zeros((r, c), dtype=np.uint8)
    
    kernel = kernel / 121
    for i in range(1, r - len(kernel)):
        for j in range(1, c - len(kernel)):
            signal_patch=img[i:i+len(kernel),j:j+len(kernel)]   
            gauss[i][j]=(kernel*signal_patch).sum()
           
    return gauss
    '''rows=signal.shape[0]-kernel.shape[0]+1
    cols=signal.shape[1]-kernel.shape[1]+1
    output=np.zeros((rows,cols), 'uint8')
    try:
                gauss[i][j]=(kernel*signal_patch).sum()
            except IndexError:
                gauss[i][j]=gauss[i][j]
                            

    kernel_reversed=np.rot90(np.rot90(kernel))

    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            #desplazando el kernel
            signal_patch=signal[i:i+len(kernel),j:j+len(kernel)]  
            print signal_patch
            output[i][j]=(kernel*signal_patch).sum()
            print output

    print signal_patch
    print output''' 


In [ ]:
from PIL import Image
import math

# img = Image.open('input.jpg').convert('L')
# r = radiuss
def gauss_blur(img, r):
    imgData = img

    bluredImg = Image.new(img.mode, img.size)
    bluredImgData = list(bluredImg.getdata())

    rs = int(math.ceil(r * 2.57))

    for i in range(0, img.height):
        for j in range(0, img.width):
            val = 0
            wsum = 0
            for iy in range(i - rs, i + rs + 1):
                for ix in range(j - rs, j + rs + 1):
                    x = min(img.width - 1, max(0, ix))
                    y = min(img.height - 1, max(0, iy))
                    dsq = (ix - j) * (ix - j) + (iy - i) * (iy - i)
                    weight = math.exp(-dsq / (2 * r * r)) / (math.pi * 2 * r * r)
                    val += imgData[y * img.width + x] * weight
                    wsum += weight 
            bluredImgData[i * img.width + j] = round(val / wsum)

    bluredImg.putdata(bluredImgData)
    return bluredImg

In [ ]:
def mygauss_fun(img,opc=0):
    if orient == 0:
        r, c = img.shape
        gauss = np.zeros((r, c), dtype=np.uint8)

        kernel= np.array([[1, 2, 1],
                         [2, 8, 2],
                         [1, 2, 1]],dtype=np.float)

        for x in range(r-1):
            for y in range(c-1):
                suma = 0
                for i in range(len(kernel)):
                    for j in range (len(kernel)):
                        try:
                            if x < r-1 or y <c-1:
                                suma += int(img[(x-1)+i,(y-1)+j] * kernel[j][i])
                        except IndexError:
                            suma += 0

                gauss[x,y] = suma
    elif orient == 1:       
        r, c = img.shape
        gauss = np.zeros((r, c), dtype=np.uint8)
        kgss = np.array([[1, 2, 1],
                         [2, 8, 2],
                         [1, 2, 1]],dtype=np.float)
        kgss = kgss / 20
        for i in range(1, r - 1):
            for j in range(1, c - 1):
                gauss[i, j] = (img[i - 1, j - 1] * kgss[0, 0] + img[i, j - 1] * kgss[0, 1] + img[i + 1, j - 1] * kgss[0, 2] \
                             + img[i - 1, j] * kgss[1, 0] + img[i, j] * kgss[1, 1] + img[i + 1, j] * kgss[1, 2] \
                             + img[i - 1, j + 1] * kgss[2, 0] + img[i, j + 1] * kgss[2, 1] + img[i + 1, j + 1] * kgss[2, 2])
           
    return gauss
def mysobel_fun(img, orient='x'):
    # Calculate directional gradient
    r,c=img.shape
    sobel=np.zeros((r,c), dtype=np.uint8)
    if orient == 'x':
        kernel=np.array([[-1,-2,-1],
                  [0, 0 ,0],
                  [1, 2 ,1]],dtype=np.float)
        kernel = kernel/4
        for i in range(1,r -1):
            for j in range(1,c -1):
                sobel[i, j] = np.sqrt((
                                        (img[i - 1, j - 1] * kernel[0, 0] + img[i, j - 1] * kernel[0, 1] + img[i + 1, j - 1] * kernel[0, 2]\
                                        + img[i - 1, j + 1] * kernel[2, 0] + img[i, j + 1] * kernel[2, 1] + img[i + 1, j + 1] * kernel[2, 2])
                )**2)
    
    elif orient == 'y':
        kernel=np.array([[-1,0,1],
                [-2,0,2],
                [-1,0,1]],dtype=np.float)
        kernel = kernel/4
        for i in range(1, r - 1):
            for j in range(1, c - 1):
                sobel[i, j] =np.sqrt((
                                           (img[i - 1, j - 1] * kernel[0, 0] + img[i + 1, j - 1] * kernel[0, 2]\
                                           + img[i - 1, j] * kernel[1, 0] + img[i + 1, j] * kernel[1, 2]\
                                           + img[i - 1, j + 1] * kernel[2, 0] + img[i + 1, j + 1] * kernel[2, 2])
                )**2)        
    return sobel


In [ ]:
def gaussian_blur(img, kernel = 5):
    '''Modificar esta función'''
    
            
            
    '''  mitad = length(kernel) / 2;'''
    
    '''* Proceso de convolucion
     * Recorro la imagen en los dos primeros for, al igual que el kernel
    
     * en la variable mm hallo el indice de la fila del kernel alrevez, al
     * igual que la variable nn, almacena la columna del kernel alrevez,
     * las variables ii,jj son para almacenar la posicion de las imagenes tomando
     * en cuenta su limite exterior es decir i-1, j-1,la variable acumulador almacena el resultado
     * que luego es asignado en la posicion de la imagen resultante
     */  
    mitad = len(kernel) / 2
    mm =0
    nn =0

    for i in range(0, r -1):
        for j in range(0, c -1):
            #Variable acumuladora
            acumulador = 0
            for m in range(0,len(kernel)):
                mm = len(kernel) - 1 - m # Indice de la fila del kernel alrevez
                for n in range(0,len(kernel)):# Columnas del kernel                
                        nn = len(kernel) - 1 - n # Indice de la columna del kernel alrevez
                        ii = i + (m - mitad)
                        jj = j + (n - mitad)
                        # validar limites de la imagen 00000
                        if ii >= 0 & ii < len(img) & jj >= 0 & jj < len(img):
                            acumulador += img[ii][jj] * kernel[mm][nn]'''
                                              
   # return cv2.GaussianBlur(img,(kernel,kernel),0)gauss_blur(img, r):mygauss_fun(img,opc=0)
    img = Mygauss_blur(img)
    return img

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    if orient == 'x':
        x,y = 1,0
    elif orient == 'y':
        x,y = 0,1
        
    '''Modificar esta función'''
    sobel = cv2.Sobel(img, cv2.CV_64F, x, y, ksize=sobel_kernel)
    
    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)

    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a mask of 1's where the scaled gradient magnitude 
    # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary

def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Take the gradient in x and y separately
    '''Modificar esta función'''
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    '''Modificar esta función'''
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Calculate the magnitude 
    abs_sobel = np.sqrt(sobelx**2 +sobely**2)

    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a binary mask where mag thresholds are met
    sbinary = np.zeros_like(scaled_sobel)
    sbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return sbinary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Take the gradient in x and y separately
    '''Modificar esta función'''
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    '''Modificar esta función'''
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)   
    abs_sobely = np.absolute(sobely)
    
    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    grad_dir = np.arctan2(abs_sobely, abs_sobelx)
    
    # Create a binary mask where direction thresholds are met
    sbinary = np.zeros_like(grad_dir)
    sbinary[(grad_dir >= thresh[0]) & (grad_dir <= thresh[1])] = 1
    return sbinary

## 1. Prueba en Imagen

In [ ]:
# Test filter on single image
im = plt.imread("BTD.jpg")

#im = plt.imread("BTD.jpg")
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

# Gaussian Blur Filtering
gray = gaussian_blur(gray, kernel = 5)

# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

# Apply each of the thresholding functions
gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(80, 100))
grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(50, 100))
mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(20, 100))
dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0, np.pi/2))

# Create combined image with sobel filters applied
combined = np.zeros_like(dir_binary)
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

rgb_combined = np.asarray(np.dstack((combined, combined, combined)), dtype=np.float64)

In [ ]:
# Plot the result
f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(10, 15))
f.tight_layout()
ax1.imshow(im)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(gray, cmap='gray')
ax2.set_title('Gray Image', fontsize=20)
ax3.imshow(rgb_combined, cmap='gray')
ax3.set_title('Combined Sobel Image', fontsize=20)
ax4.imshow(gradx, cmap='gray')
ax4.set_title('GradX Image', fontsize=20)
ax5.imshow(grady, cmap='gray')
ax5.set_title('GradY Image', fontsize=20)
ax6.imshow(((mag_binary == 1) & (dir_binary == 1)), cmap='gray')
ax6.set_title('Thresholded Grad. Dir.', fontsize=20)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show() 

## 2. Prueba en Videos

In [ ]:
def process_image(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = gaussian_blur(gray, kernel = 5)
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gray, orient='x', sobel_kernel=ksize, thresh=(50, 100))
    grady = abs_sobel_thresh(gray, orient='y', sobel_kernel=ksize, thresh=(50, 100))
    mag_binary = mag_thresh(gray, sobel_kernel=ksize, mag_thresh=(20, 100))
    dir_binary = dir_threshold(gray, sobel_kernel=ksize, thresh=(0, np.pi/4))
    
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    
    
    rgb_combined = np.asarray(np.dstack((combined, combined, combined)), dtype=np.uint8)
    
    retval, threshold = cv2.threshold(rgb_combined, 0, 255, cv2.THRESH_BINARY)
    
    return threshold

In [ ]:
filter_output = 'Batman_Filter.mp4'
clip1 = VideoFileClip("Batman.mp4")
batman_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time batman_clip.write_videofile(filter_output, audio=False)

In [ ]:
HTML("""
<video width="720" height="480" controls>
  <source src="{0}">
</video>
""".format(filter_output))

In [ ]:
cap = cv2.VideoCapture(0)  # 0 for Python 3, 1 for Python 2. No idea why?!!

out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20,(640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,180)
        new_frame = process_image(frame)
        out.write(new_frame)

        cv2.imshow('frame',new_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()